In [7]:
import streamlit as st
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [8]:
# Firebase configuration
collection_name = "filmsDashboard"
cred = credentials.Certificate("./secret/credentials.json")

# Load dataset
df_to_import = pd.read_csv("./dataset/movies.csv")


In [9]:
df_to_import.head()


,company,director,genre,name
0,Columbia Pictures Corporation,Rob Reiner,Adventure,Stand by Me
1,Paramount Pictures,John Hughes,Comedy,Ferris Bueller's Day Off
2,Paramount Pictures,Tony Scott,Action,Top Gun
3,Twentieth Century Fox Film Corporation,James Cameron,Action,Aliens
4,Walt Disney Pictures,Randal Kleiser,Adventure,Flight of the Navigator


In [10]:

# Function definitions
def is_firebase_initialized():
    """Checks if a Firebase app has been initialized.

    Returns:
        True if a Firebase app has been initialized, False otherwise.
    """
    try:
        # Attempt to access the default app. If it exists, it's initialized.
        firebase_admin.get_app()
        return True
    except ValueError:
        return False
    except Exception as e:
        # Handle other potential exceptions
        print(f"Error checking Firebase initialization: {e}")
        return False


In [11]:

def import_to_firebase(df, collection_name):
    try:
        collection_list = [collection.id for collection in db.collections()]
        if collection_name not in collection_list:
            for index, row in df.iterrows():
                # Convert each row to a dictionary
                row_dict = row.to_dict()
                # Create a new document in the collection
                db.collection(collection_name).add(row_dict)
            print(
                f'DataFrame importado correctamente a la colección "{collection_name}"'
            )
        else:
            print(
                f'La colección "{collection_name}" ya existe. No se importó el DataFrame.'
            )

    except Exception as e:
        print(f"Error al importar DataFrame: {e}")



In [12]:

def read_database_to_dataframe(collection_name) -> pd.DataFrame:
    """Lee todos los documentos de una colección de Firestore y los devuelve como un DataFrame de Pandas."""
    try:
        docs = db.collection(collection_name).stream()
        data = []
        for doc in docs:
            data.append(doc.to_dict())
        df = pd.DataFrame(data)
        return df
    except Exception as e:
        print(f"Error al leer datos de Firestore: {e}")
        return None


In [13]:

# Initialize Firebase
if not is_firebase_initialized():
    try:
        firebase_admin.initialize_app(cred)
        print("Firebase app initialized.")
    except Exception as e:
        print(f"Error initializing Firebase app: {e}")
else:
    print("Firebase app already initialized.")

db = firestore.client()

Firebase app already initialized.


In [14]:

# Import data to Firebase
import_to_firebase(df_to_import, collection_name)

# Read data from Firebase
df_films = read_database_to_dataframe(collection_name)

if df_films is not None:
    print("DataFrame base cargado correctamente.")
    print(df_films.head())
else:
    print("No se pudo cargar el DataFrame base.")

# Streamlit app
st.title("Netflix app")

# Initialize variables for storing results
filter_results = df_films.copy()  # Initialize with all data
search_results = None


# Sidebar for filters
with st.sidebar:
    st.header("Filtros")

    # Checkbox to show all films
    show_all = st.checkbox("Mostrar todos los filmes", value=True)

    # Text input to search films
    film_title = st.text_input("Titulo del filme:")

    # Button to search films
    if st.button("Buscar filmes"):
        if film_title:
            search_results = df_films[
                df_films["name"].str.contains(film_title, case=False)
            ]
        else:
            search_results = df_films.copy()

    # Selector to select director
    directors = df_films["director"].unique().tolist()
    selected_director = st.selectbox("Seleccionar Director", ["Todos"] + directors)

    # Button to filter director
    if st.button("Filtrar director"):
        if selected_director != "Todos":
            filter_results = df_films[df_films["director"] == selected_director]
        else:
            filter_results = df_films.copy()

    # Section to add new film (outside sidebar)
    st.subheader("Nuevo filme")
    new_name = st.text_input("Name:")
    new_company = st.selectbox("Company", df_films["company"].unique().tolist())
    new_director = st.text_input("Director:")

# Apply filters
final_results = filter_results.copy()
if search_results is not None:
    final_results = final_results.merge(search_results, how="inner")

# Show results on the main page
if show_all:
    st.write(f"Lista de filmes")
    st.write(f"Mostrando: {len(final_results)} films de {len(df_films)}")
    st.write(final_results)


La colección "filmsDashboard" ya existe. No se importó el DataFrame.


2025-03-09 08:24:59.363 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.520 
  command:

    streamlit run /home/feldt/.local/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-09 08:24:59.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.524 Thread 'MainThread

DataFrame base cargado correctamente.
                  name                        company         director  \
0         Major League             Mirage Enterprises    David S. Ward   
1       Stars and Bars  Columbia Pictures Corporation     Pat O'Connor   
2      Rumpelstiltskin              Cannon Group, The     David Irving   
3       Dark Obsession        Film Four International  Nick Broomfield   
4  She's Having a Baby             Paramount Pictures      John Hughes   

     genre  
0   Comedy  
1   Comedy  
2  Fantasy  
3    Drama  
4   Comedy  


2025-03-09 08:24:59.570 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 08:24:59.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
